In [4]:
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures
import glob
import os

# --- Simulation Parameters ---
NUM_SYSTEMS = 24
PARTICLES_PER_SYSTEM = 2048
BOX_MIN = -1.0
BOX_MAX = 1.0
SYSTEM_TO_PLOT = 0
OUTPUT_DIR = "frames"

os.makedirs(OUTPUT_DIR, exist_ok=True)

particle_dtype = np.dtype([
    ('position', np.float32, 2),
    ('velocity', np.float32, 2),
    ('acceleration', np.float32, 2)
])

files = sorted(glob.glob("build/dat/*_step.bin"))
if not files:
    print("Error: No .bin files found in the 'dat' directory.")
    exit()

def render_frame(idx_file):
    """Render one frame and save as PNG."""
    idx, filename = idx_file
    flat_data = np.fromfile(filename, dtype=particle_dtype)
    structured_data = flat_data.reshape((NUM_SYSTEMS, PARTICLES_PER_SYSTEM))
    positions = structured_data[SYSTEM_TO_PLOT]['position']
    velocities = structured_data[SYSTEM_TO_PLOT]['acceleration']

    speed = np.linalg.norm(velocities, axis=1)
    speed_norm = (speed - speed.min()) / (speed.max() - speed.min() + 1e-8)

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(BOX_MIN, BOX_MAX)
    ax.set_ylim(BOX_MIN, BOX_MAX)
    ax.set_aspect('equal')

    ax.scatter(
        positions[:, 0], positions[:, 1],
        s=5, c=speed_norm, cmap='gist_ncar'
    )

    ax.set_title(f"Particle Simulation (System {SYSTEM_TO_PLOT}, Frame {idx})")

    out_path = os.path.join(OUTPUT_DIR, f"frame_{idx:05d}.png")
    plt.savefig(out_path, dpi=150)
    plt.close(fig)
    return out_path

# --- Parallel frame rendering ---
print(f"Rendering {len(files)} frames using multithreading...")
with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    list(executor.map(render_frame, enumerate(files)))

print(f"All frames saved in '{OUTPUT_DIR}' directory.")


Rendering 999 frames using multithreading...
All frames saved in 'frames' directory.
